In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/CAR DETAILS FROM CAR DEKHO.csv')

# Some exploratory analysis

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
plt.title('Number of cars based on type of Fuel');
sns.countplot(data=df, x='fuel');

In [ ]:
plt.title('Number of cars based on type of Seller');
sns.countplot(data=df, x='seller_type');

In [ ]:
plt.title('Number of cars based on type of Transmission');
sns.countplot(data=df, x='transmission');

In [ ]:
plt.title('Number of cars based on type of Owners');
sns.countplot(data=df, x='owner');
plt.xticks(rotation=90);

In [ ]:
#removing name from dataset since it's a unique identifier for each row
df = df.drop('name',axis=1)

In [ ]:
#changing categorical to dummy variables
df1 = pd.get_dummies(data=df, drop_first=True)

In [ ]:
df1

In [ ]:
#checking the correlation with the label
df1.corr()['selling_price']

In [ ]:
plt.figure(figsize=(12,8),dpi=150);
plt.title('Correlation between different variables')
sns.heatmap(data=df1.corr(),annot=True);

In [ ]:
sns.scatterplot(data=df1, x='year',y='selling_price');

In [ ]:
#turning year to age
df1['year'] = df1['year'].apply(lambda year: 2021 - year)

In [ ]:
sns.scatterplot(data=df1, x='year',y='selling_price');

In [ ]:
sns.displot(data=df1, x='selling_price');
plt.xlim(0,500000)

In [ ]:
sns.scatterplot(data=df, x='km_driven', y='selling_price',alpha=0.5)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df1.drop('selling_price',1)

In [ ]:
y = df1['selling_price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_X_train = scaler.fit_transform(X_train)

In [ ]:
scaled_X_test = scaler.transform(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# compare different models

In [ ]:
def report_model(model,X_train,y_train,X_test,y_test):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test,y_pred)
    mse = mean_squared_error(y_test,y_pred)
    print(mae)
    print(np.sqrt(mse))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
report_model(lr,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge()

In [ ]:
report_model(ridge,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso()

In [ ]:
report_model(lasso,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
ridgecv = RidgeCV()

In [ ]:
report_model(ridgecv,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dtr = DecisionTreeRegressor()

In [ ]:
report_model(dtr,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

In [ ]:
rfr = RandomForestRegressor()

In [ ]:
report_model(rfr,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
abc = AdaBoostRegressor()

In [ ]:
report_model(abc,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
# Choosing RandomForest as the model
# Using GridSearchCV to find the best hyperparameters

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params = {
            "n_estimators": np.linspace(100, 1200, 12).astype(int),
            "max_depth": [5, 7, 9, 10, 15, 20, 25, 30, 35, 40],
            "min_samples_split": [2, 4, 6, 8, 10, 15, 20],
            "min_samples_leaf": [i for i in range(1, 11)]}

In [ ]:
model = RandomizedSearchCV(rfr,params)

In [ ]:
model.fit(scaled_X_train,y_train)

In [ ]:
model.best_params_

In [ ]:
y_pred =model.predict(scaled_X_test)

In [ ]:
mean_absolute_error(y_test,y_pred)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
plt.figure('Residuals plot')
sns.residplot(x=y_pred, y=y_test);

# Finalizing the model

In [ ]:
final_model = RandomForestRegressor(n_estimators=300, min_samples_split=10, min_samples_leaf=3, max_depth=15)

In [ ]:
final_model.fit(X,y)

In [ ]:
importance = pd.Series(final_model.feature_importances_,index=X.columns).sort_values(ascending=False)

In [ ]:
importance.plot(kind='bar',xlabel='Features',ylabel='Coefficient',title='Importance of each feature',colormap='Accent');
;

In [ ]:
#saving the model
import pickle

file = open('car_pred.pkl', 'wb')

pickle.dump(final_model, file)